

# 종로구 미세먼지 데이터 정제

In [5]:
# 종로구 동별 파일 불러오기

import glob
import os
import pandas as pd

os.getcwd()

filenames = glob.glob('../data/weatherdata/JONGNO/*.csv')

In [6]:
# 종로구 동별 파일 리스트로

filenames

['../data/weatherdata/JONGNO\\V10O1610252.csv',
 '../data/weatherdata/JONGNO\\V10O1610540.csv',
 '../data/weatherdata/JONGNO\\V10O1610542.csv',
 '../data/weatherdata/JONGNO\\V10O1610543.csv',
 '../data/weatherdata/JONGNO\\V10O1610544.csv',
 '../data/weatherdata/JONGNO\\V10O1610545.csv',
 '../data/weatherdata/JONGNO\\V10O1610546.csv',
 '../data/weatherdata/JONGNO\\V10O1610567.csv',
 '../data/weatherdata/JONGNO\\V10O1611145.csv',
 '../data/weatherdata/JONGNO\\V10O1611151.csv',
 '../data/weatherdata/JONGNO\\V10O1611170.csv',
 '../data/weatherdata/JONGNO\\V10O1611172.csv',
 '../data/weatherdata/JONGNO\\V10O1611173.csv',
 '../data/weatherdata/JONGNO\\V10O1611220.csv',
 '../data/weatherdata/JONGNO\\V10O1611251.csv',
 '../data/weatherdata/JONGNO\\V10O1611255.csv',
 '../data/weatherdata/JONGNO\\V10O1611258.csv',
 '../data/weatherdata/JONGNO\\V10O1611623.csv',
 '../data/weatherdata/JONGNO\\V10O1611634.csv',
 '../data/weatherdata/JONGNO\\V10O1611639.csv',
 '../data/weatherdata/JONGNO\\V10O161164

In [8]:
# 진행 결과 확인을 위해 i 설정
i = 0

# 동 하나씩 불러오기
for f in filenames :
    aa = pd.read_csv(f)
    
    # 필요한 행만 가져오기
    aa = aa[['tm','serial','pm10','pm25']]
    
    # 미세먼지 농도값이 -999(결측치)가 아닐 경우, 500보다 낮을 경우만 가져오기
    aa = aa[aa['pm10'] != -999]
    aa = aa[aa['pm10'] < 500]
    
    # 추후 평균계산 작업을 위해 날짜 데이터 타입을 str으로 변경하고 일자별로
    aatm = list(aa['tm'])
    aatm = [str(t)[:8] for t in aatm]
    aa['tm'] = aatm
    
    # 추후 평균계산 작업을 위해 컬럼명 통일하고 인덱스 리셋
    aa = aa.rename(columns = {'tm': 'Date'})
    aa = aa.reset_index(drop=True)
    
    # 피봇테이블 이용해서 날짜별 미세먼지, 초미세먼지 평균계산
    mean_pm = aa.pivot_table(values=['pm10','pm25'],index = 'Date', aggfunc='mean')
    
    # 날짜가 인덱스에 들어가 있으므로 별개의 행으로 분리하는 작업
    pmin = list(mean_pm.index)
    mean_pm['Date'] = pmin
    mean_pm = mean_pm.reset_index(drop=True)
    mean_pm = mean_pm[['Date','pm10', 'pm25']]
    
    # 미세먼지, 초미세먼지 농도값 각각 소수점 세 자리까지
    
    pmro10 = list(mean_pm['pm10'])
    pmro10 = [round(ro10,3) for ro10 in pmro10]
    mean_pm['pm10'] = pmro10
    
    pmro25 = list(mean_pm['pm25'])
    pmro25 = [round(ro25,3) for ro25 in pmro25]
    mean_pm['pm25'] = pmro25 
    
    # 뉴스기사 개수 데이터 불러오기 
    cntnews = pd.read_csv('../data/COUNT_T_MERGE2.csv')
    cntnews = cntnews[['DATE','COUNT_T']]
    cntnews = cntnews.rename(columns = {'DATE': 'Date'})
    
    # 2018년 4월 1일부터 2019년 3월 31일까지 리스트로 만들기 (날짜 리스트)
    dt_list = pd.date_range("2018-4-1", "2019-3-31")
    dt_list2 = list(dt_list)
    dt_list2 = [str(dt) for dt in dt_list2]
    dt2 = [t[:4]+t[5:7]+t[8:10] for t in dt_list2]
    
    # 일자별 미세먼지 농도와 뉴스기사 개수 날짜를 각각 리스트로 만들기
    tes1 = list(mean_pm['Date'])
    tes2 = list(cntnews['Date'])
    tes2 = [str(t) for t in tes2]
    cntnews['Date'] = tes2
    
    # 날짜 리스트에 없는 미세먼지 농도 해당일자 출력
    for d in dt2 :
        if d in tes1 :
            continue
        else :
            print(d)
    print('----------------------------------------------')
    
    # 날짜 리스트에 없는 뉴스기사 개수 해당일자 출력
    for d in dt2 :
        if d in tes2 :
            continue
        else :
            print(d)
    print('----------------------------------------------') 
    
    # 미세먼지 데이터와 뉴스기사 개수 데이터 merge 
    results = mean_pm.merge(cntnews, on='Date', how='outer')  
    
    # 날짜 개수 불일치로 인한 결측치 제거
    results = results.dropna()
    
    results.to_csv('../data/weatherdata/'+f[27:-4]+'_dust.csv',index=False)
    
    i += 1
    print(f[27:-4]+'--'+str(i)+'번째 완료했습니다.')
    print('==============================================')

20180923
20180924
20180925
20180926
20180927
20180928
20180929
20180930
20181001
20181002
20181003
20181004
20181005
20181006
20181007
20181008
20181009
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1610252--1번째 완료했습니다.
20180704
20180705
20180706
20180707
20180708
20180709
20180710
20180711
20180712
20180713
20180714
20180715
20180716
20180717
20180718
20180719
20180720
20180721
20180722
20180723
20180724
20180725
20180726
20180727
20180728
20180729
20180730
20180731
20180801
20180802
20180803
20180804
20180805
20180806
20180807
20180808
20180809
20180810
20180811
20180812
20180813
20180814
20180815
20180816
20180817
20180818
20180819
20180820
20180821
20180822
20180823
20180824
20180825
20180826
20180827
20180828
20180829
20180830
20180831
20180901
20180902
20180903
20180904
20180905
20180906

----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1611170--11번째 완료했습니다.
20181228
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1611172--12번째 완료했습니다.
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1611173--13번째 완료했습니다.
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
--------

----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1611722--25번째 완료했습니다.
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1611750--26번째 완료했습니다.
20181027
20181028
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1612106--27번째 완료했습니다.
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325

# 노원구 미세먼지 데이터 정제

In [9]:
# 노원구 동별 파일 불러오기

import glob
import os
import pandas as pd

os.getcwd()

filenames = glob.glob('../data/weatherdata/NOWON/*.csv')

In [10]:
filenames

['../data/weatherdata/NOWON\\V10O1610102.csv',
 '../data/weatherdata/NOWON\\V10O1610200.csv',
 '../data/weatherdata/NOWON\\V10O1610293.csv',
 '../data/weatherdata/NOWON\\V10O1610297.csv',
 '../data/weatherdata/NOWON\\V10O1610312.csv',
 '../data/weatherdata/NOWON\\V10O1610351.csv',
 '../data/weatherdata/NOWON\\V10O1610356.csv',
 '../data/weatherdata/NOWON\\V10O1610376.csv',
 '../data/weatherdata/NOWON\\V10O1610610.csv',
 '../data/weatherdata/NOWON\\V10O1610616.csv',
 '../data/weatherdata/NOWON\\V10O1610629.csv',
 '../data/weatherdata/NOWON\\V10O1610630.csv',
 '../data/weatherdata/NOWON\\V10O1610642.csv',
 '../data/weatherdata/NOWON\\V10O1610643.csv',
 '../data/weatherdata/NOWON\\V10O1611097.csv',
 '../data/weatherdata/NOWON\\V10O1611100.csv',
 '../data/weatherdata/NOWON\\V10O1611102.csv',
 '../data/weatherdata/NOWON\\V10O1611104.csv',
 '../data/weatherdata/NOWON\\V10O1611150.csv',
 '../data/weatherdata/NOWON\\V10O1611229.csv',
 '../data/weatherdata/NOWON\\V10O1611652.csv',
 '../data/wea

In [11]:
# 진행 결과 확인을 위해 i 설정
i = 0

# 동 하나씩 불러오기
for f in filenames :
    aa = pd.read_csv(f)
    
    # 필요한 행만 가져오기
    aa = aa[['tm','serial','pm10','pm25']]
    
    # 미세먼지 농도값이 -999(결측치)가 아닐 경우, 500보다 적을 경우 제거
    aa = aa[aa['pm10'] != -999]
    aa = aa[aa['pm10'] < 500]
    
    # 추후 평균계산 작업을 위해 날짜 데이터 타입을 str으로 변경하고 일자별로
    aatm = list(aa['tm'])
    aatm = [str(t)[:8] for t in aatm]
    aa['tm'] = aatm
    
    # 추후 평균계산 작업을 위해 컬럼명 통일하고 인덱스 리셋
    aa = aa.rename(columns = {'tm': 'Date'})
    aa = aa.reset_index(drop=True)
    
    # 피봇테이블 이용해서 날짜별 미세먼지, 초미세먼지 평균계산
    mean_pm = aa.pivot_table(values=['pm10','pm25'],index = 'Date', aggfunc='mean')
    
    # 날짜가 인덱스에 들어가 있으므로 별개의 행으로 분리하는 작업
    pmin = list(mean_pm.index)
    mean_pm['Date'] = pmin
    mean_pm = mean_pm.reset_index(drop=True)
    mean_pm = mean_pm[['Date','pm10', 'pm25']]
    
    # 미세먼지, 초미세먼지 농도값 각각 소수점 세 자리까지
    
    pmro10 = list(mean_pm['pm10'])
    pmro10 = [round(ro10,3) for ro10 in pmro10]
    mean_pm['pm10'] = pmro10
    
    pmro25 = list(mean_pm['pm25'])
    pmro25 = [round(ro25,3) for ro25 in pmro25]
    mean_pm['pm25'] = pmro25 
    
    # 뉴스기사 개수 데이터 불러오기 
    cntnews = pd.read_csv('../data/COUNT_T_MERGE2.csv')
    cntnews = cntnews[['DATE','COUNT_T']]
    cntnews = cntnews.rename(columns = {'DATE': 'Date'})
    
    # 2018년 4월 1일부터 2019년 3월 31일까지 리스트로 만들기 (날짜 리스트)
    dt_list = pd.date_range("2018-4-1", "2019-3-31")
    dt_list2 = list(dt_list)
    dt_list2 = [str(dt) for dt in dt_list2]
    dt2 = [t[:4]+t[5:7]+t[8:10] for t in dt_list2]
    
    # 일자별 미세먼지 농도와 뉴스기사 개수 날짜를 각각 리스트로 만들기
    tes1 = list(mean_pm['Date'])
    tes2 = list(cntnews['Date'])
    tes2 = [str(t) for t in tes2]
    cntnews['Date'] = tes2
    
    # 날짜 리스트에 없는 미세먼지 농도 해당일자 출력
    for d in dt2 :
        if d in tes1 :
            continue
        else :
            print(d)
    print('----------------------------------------------')
    
    # 날짜 리스트에 없는 뉴스기사 개수 해당일자 출력
    for d in dt2 :
        if d in tes2 :
            continue
        else :
            print(d)
    print('----------------------------------------------') 
    
    # 미세먼지 데이터와 뉴스기사 개수 데이터 merge 
    results = mean_pm.merge(cntnews, on='Date', how='outer')  
    
    # 날짜 개수 불일치로 인한 결측치 제거
    results = results.dropna()
    
    results.to_csv('../data/weatherdata/'+f[26:-4]+'_dust.csv',index=False)
    
    i += 1
    print(f[26:-4]+'--'+str(i)+'번째 완료했습니다.')
    print('==============================================')

20180401
20180402
20180403
20180404
20180405
20180406
20180407
20180408
20180409
20180410
20180411
20180412
20180413
20180414
20180415
20180416
20180417
20180418
20180419
20180420
20180421
20180422
20180423
20180424
20180425
20180426
20180427
20180428
20180429
20180430
20180501
20180502
20180503
20180504
20180505
20180506
20180508
20180509
20180916
20180918
20180919
20180920
20180921
20180922
20180923
20180924
20180925
20180926
20180927
20180928
20180929
20180930
20181001
20181002
20181003
20181004
20181005
20181006
20181007
20181008
20181009
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1610102--1번째 완료했습니다.
20180502
20180504
20180505
20180506
20180508
20180509
20180518
20180519
20180521
20180522
20180523
20180524
20180525
20180526
20180527
20180528
20180529
20180530
20180531
20180601
20180602

20180401
20180402
20180403
20180404
20180405
20180406
20180407
20180408
20180410
20180411
20180412
20180413
20180414
20180415
20180416
20180417
20180418
20180419
20180421
20180422
20180428
20180429
20180501
20180504
20180505
20180506
20180507
20180509
20180510
20180511
20180512
20180513
20180514
20180515
20180516
20180517
20180518
20180519
20180520
20180521
20180522
20180523
20180524
20180525
20180526
20180527
20180528
20180529
20180530
20180531
20180601
20180602
20180603
20180604
20180605
20180606
20180607
20180608
20180609
20180610
20180611
20180612
20180613
20180614
20180615
20180616
20180617
20180618
20180619
20180620
20180621
20180622
20180623
20180624
20180625
20180626
20180627
20180628
20180629
20180630
20180701
20180702
20180703
20180704
20180705
20180706
20180707
20180708
20180709
20180710
20180711
20180712
20180713
20180714
20180715
20180716
20180717
20180718
20180719
20180720
20180721
20180722
20180723
20180724
20180725
20180726
20180727
20180728
20180729
20180730
20180731
2

20180920
20180921
20180922
20180923
20180924
20180925
20180926
20180927
20180928
20180929
20180930
20181001
20181002
20181003
20181004
20181005
20181006
20181007
20181008
20181009
----------------------------------------------
20180415
20180529
20180601
20190109
20190123
20190124
20190207
20190208
20190216
20190222
20190223
20190308
20190309
20190310
20190318
20190325
----------------------------------------------
V10O1610642--13번째 완료했습니다.
20180630
20180701
20180702
20180703
20180704
20180705
20180706
20180707
20180708
20180709
20180710
20180711
20180712
20180713
20180714
20180715
20180716
20180717
20180718
20180719
20180720
20180721
20180722
20180723
20180724
20180725
20180726
20180727
20180728
20180729
20180730
20180731
20180801
20180802
20180803
20180804
20180805
20180806
20180807
20180808
20180809
20180810
20180811
20180812
20180813
20180814
20180815
20180816
20180817
20180818
20180819
20180820
20180821
20180822
20180823
20180824
20180825
20180826
20180827
20180828
20180829
2018083